In [117]:
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.document_converter import (
    DocumentConverter,
    PdfFormatOption,
    WordFormatOption,
)
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend

from docling_core.types.doc.document import TextItem, TableItem, PictureItem, ImageRefMode

import pprint

## Default initialization still works as before:
# doc_converter = DocumentConverter()


# previous `PipelineOptions` is now `PdfPipelineOptions`
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True
#...

## Custom options are now defined per format.
doc_converter = (
    DocumentConverter(  # all of the below is optional, has internal defaults.
        allowed_formats=[
            InputFormat.PDF,
            InputFormat.IMAGE,
            InputFormat.DOCX,
            InputFormat.HTML,
            InputFormat.PPTX,
        ],  # whitelist formats, non-matching files are ignored.
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, # pipeline options go here.
                backend=PyPdfiumDocumentBackend # optional: pick an alternative backend
            ),
            InputFormat.DOCX: WordFormatOption(
                pipeline_cls=SimplePipeline # default for office formats and HTML
            ),
        },
    )
)

In [118]:
from docling.datamodel.document import ConversionResult
## Convert a single file (from URL or local path)
# conv_result: ConversionResult = doc_converter.convert("./sample_pdf/trm_proper.pdf") # previously `convert_single`
conv_result: ConversionResult = doc_converter.convert("./sample_pdf/docling_technical_report.pdf")

# ## Convert several files at once:

# input_files = [
#     "tests/data/html/wiki_duck.html",
#     "tests/data/docx/word_sample.docx",
#     "tests/data/docx/lorem_ipsum.docx",
#     "tests/data/pptx/powerpoint_sample.pptx",
#     "tests/data/2305.03393v1-pg9-img.png",
#     "tests/data/pdf/2206.01062.pdf",
# ]

# # Directly pass list of files or streams to `convert_all`
# conv_results_iter = doc_converter.convert_all(input_files) # previously `convert`

In [119]:
import pandas as pd

"""
https://ds4sd.github.io/docling/reference/docling_document/#docling_core.types.doc.DoclingDocument
"""

docling_document = conv_result.document
# Print Out Docling Document Contents
# print(conv_result.document.texts)
# print([t.text for t in conv_result.document.texts])

# print(conv_result.document.tables)
# print(conv_result.document.pictures)


# print(docling_document.print_element_tree())

## Iterate the elements in reading order, including hierachy level:
# for item, level in conv_result.document.iterate_items():
#     print(item)
#     # if not isinstance(item, PictureItem):
#     # #     print(type(item))
#     #     print(item.text)
#     # if isinstance(item, TextItem):
#     #     print("<<< TEXT >>>")
#     #     print(str(item.parent.get_ref()))
#     #     print(f".text: \n{item.text}")
#     # #     # print(f".children: \n{item.children}")
#     # elif isinstance(item, TableItem):
#     #     table_df: pd.DataFrame = item.export_to_dataframe()
#     #     print("<<< TABLE >>>")
#     #     print(table_df.head())
#     # if isinstance(item, PictureItem):
#     #     for item in item.children:
#     #         # print(type(item))
#     #         # print(type(item.get_ref()))
#     #         if "texts" in str(item.get_ref()):
#     #             print(item.resolve())

In [120]:
from pathlib import Path

# json_output_file=Path("./output/trm_docling.json")
json_output_file=Path("./output/docling_paper.json")
docling_document.save_as_json(json_output_file, image_mode=ImageRefMode.EMBEDDED)
#文字化け

# html_output_file=Path("./output/trm_docling.html")
# docling_document.save_as_html(html_output_file, image_mode=ImageRefMode.EMBEDDED)

In [ ]:
import json

# with open("./output/trm_docling.html", "w", encoding="utf-8") as f:
#     json.dump(docling_document.export_to_dict(), f, ensure_ascii=False, indent=2)
# UnicodeエスケープされたJSON文字列の例
# escaped_json = '{"message": "\\u65e5\\u672c\\u8a9e"}'

# # JSON文字列をPythonのオブジェクトに変換（この時点でUnicodeエスケープはデコードされる）
# data = json.loads(docling_document.export_to_dict())

# # Pythonオブジェクトを再度JSON形式に変換する際、ensure_ascii=Falseで日本語がそのまま出力されるようにする
# formatted_json = json.dumps(data, ensure_ascii=False, indent=2)
# print(formatted_json)



""""
以下のライブラリコードを修正し文字化け修正：
"C:\Users\user\.conda\envs\pdf_ext\Lib\site-packages\docling_core\types\doc\document.py"
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 226-227: truncated \UXXXXXXXX escape (2844947954.py, line 27)